<a href="https://colab.research.google.com/github/ewilson2023/History-Name-Mashup-Generator/blob/main/HistoryChalengeNameGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this
A "fantasy" name generator. Picks syllables of names from Ancient Egypt, Ancient Greece, Ancient Rome, and Ancient Near East and generates a mashup name

## How to use it
1) if the table of contents isn't visible, click the hamburger button at the top of the left vertical menu bar.
2) Click "Main".
3) If on mobile, close the table of contents to see the code
4) At the top of the page, you'll see a menu with options for "Commands", "Code", "Text", and "Run all".
5) Click "Run all".
6) To generate again, click the play button under the "Main" label.

# Classes and Utilities

In [15]:
# class for each cultur, storing sorted dictionaries

class Culture:
  def __init__(self, p, m, s, name):
    self.name = name
    self.pre = sort(p)
    self.mid = sort(m)
    self.suf = sort(s)




*   dicts lack explicit structure
* TypedDict: a fixed-shape dictionary where the keys are always the same bucket names, and each key always maps to the same kind of value


   

In [16]:
from typing import TypedDict

# takes the form
# "name":   list["string"],
# "name":   list[int],
"""
SortBuckets is not a class at runtime. A SortedBuckets b
is a dictionary that has specific keys and value types.
b["all"] is a list of strings,
b["c_any"] is a list of ints
"""
class SortBuckets(TypedDict):
    all: list[str]
    vv: list[int]; cc: list[int]
    vc: list[int]; cv: list[int]
    c: list[int];  v: list[int]
    c_any: list[int]; v_any: list[int]
    any_c: list[int]; any_v: list[int]

In [17]:
# return a dictionary sorted by consonant/vowel ending
def sort(list: list[str]) -> dict:
    dict = {
          "all" :[],
          "vv": [], "cc": [],
          "vc": [], "cv": [],
          "c": [],  "v": [],

          "c_any": [], "v_any": [],
          "any_c": [], "any_v": [],
      }
    for idx, str in enumerate(list):
        dict["all"].append(str)

        first = str[0]
        last = str[len(str) - 1]

        if len(str) == 1:
          if isConsonant(first) and first != "y":
            dict["c"].append(idx)
            dict["cc"].append(idx)
            dict["c_any"].append(idx)
            dict["any_c"].append(idx)
          else:
            dict["v"].append(idx)
            dict["vv"].append(idx)
            dict["v_any"].append(idx)
            dict["any_v"].append(idx)
          continue

        else:
          if isConsonant(first) or (first == "y" and isVowel(str[1])):
            dict["c_any"].append(idx)
            if isConsonant(last) or (str[len(str)-2]+last) == "qu":
              dict["cc"].append(idx)
              dict["any_c"].append(idx)
            else:
              dict["cv"].append(idx)
              dict["any_v"].append(idx)
          else:
            dict["v_any"].append(idx)
            if isConsonant(last) or (str[len(str)-2]+last) == "qu":
              dict["vc"].append(idx)
              dict["any_c"].append(idx)
            else:
              dict["vv"].append(idx)
              dict["any_v"].append(idx)


    return dict


In [18]:
def isVowel(char):
  vowels = ["a", "e", "i", "o", "u", "A", "E", "I", "O", "U"]

  if char in vowels:
    return True
  else:
    return False

def isConsonant(str):
  return not isVowel(str)

In [19]:
import random

##################### v2
def generateName(cultures: list[Culture]) -> None:
  """ Pick an index into b["all"], using fallback logic:
      1) try the preferred index-bucket (e.g., "c_any")
      2) if empty, try the other bucket (e.g., "v_any")
      3) if still empty, fall back to ANY valid index from b["all"]
    Returns: an int index suitable for b["all"][idx] """
  def pick_idx(
      b: SortBuckets, prefer: str, other: str) -> int:
        pool: list[int] = b[prefer]     # ex. b = [0, 5, 8] (idxs)
        if not pool:                    # if empty, pick from other list of idxs
            pool = b[other]             # still a list[int]
        if not pool:                    # if still empty, pick from all
            pool = list(range(len(b["all"])))
        return random.choice(pool)      # return random idx int from the pool

  def pick_syllable(
      cult: Culture,
      next: str, part_choice: str ) -> str:
    sbl: str
    idx: int

    if part_choice == "prefix":
      word_part = cult.pre
    elif part_choice == "middle":
      word_part = cult.mid
    else: # part == "suffix"
      word_part = cult.suf

    # choose what pool to get the index from
    if next == 'c':
      idx = pick_idx(word_part, "c_any", "v_any")
    else: # next == 'v'
      idx = pick_idx(word_part, "v_any", "c_any")

    # choose the syllable associated with this index
    sbl = word_part.get("all")[idx]

    # if a prefix was chosen in fallback, make the
    # first letter lowercase (hypen aware)
    if part_choice != "prefix" and sbl[0].isalpha() and sbl[0].isupper():
      sbl[0] = sbl[0].lower()

    return sbl


  """############### PREFIX ###############"""
  name: str
  cult: Culture = random.choice(cultures)

  # prefix: random of all this cultures's prefixes
  p: str = random.choice(cult.pre.get("all"))

  debug1: str = (cult.name)
  debug2: str = (p)

  name = p;   """add it to name"""

  if isVowel(p[len(p) - 1]):
    next = 'c'
  else:
    next = 'v'

  """############### MIDDLE ###############"""
  m: str = ""
  if random.choice([True, False]):
    # choose a new culture
    cult = random.choice(cultures)
    m = pick_syllable(cult, next, "middle")

    if m != "":
      if isVowel(m[len(m) - 1]):
        next = 'c'
      else:
        next = 'v'

      """add it to name"""
      name = name + m
      debug1 = (debug1+ " - " + cult.name)
      debug2 = (debug2+ " - " + m)

    # 25% chance of second mid syllable, same culture
    if random.choice([True, False]) and len(name) < 8:
      m2 = pick_syllable(cult, next, "middle")

      if m2 != "":
        if isVowel(m2[len(m2) - 1]):
          next = 'c'
        else:
          next = 'v'

        """add it to name"""
        name = name + m2
        debug1 = (debug1+ " - " + cult.name)
        debug2 = (debug2+ " - " + m2)


  """############### SUFFIX ###############"""
  # choose a new culture
  cult = random.choice(cultures)
  s = pick_syllable(cult, next, "middle")

  """add it to name"""
  if s != "":
    name = name + s
    debug1 = (debug1+ " - " + cult.name)
    debug2 = (debug2+ " - " + s)



  """############### OUTPUT ###############"""
  print(name)
  print("  "+debug1)
  print("  "+debug2 + "\n")




In [20]:
# list syllables for all cultures
#### EGYPTIAN ####
e_p = ["A","Nu", "Ha", "Dje", "Djo", "Amen", "Amon", "Anu", "Bas",
        "Sob", "Nef", "Nefer", "Sen", "Ahm", "Akhe", "Khu", "Kh",
        "Khn", "Kha", "Ram", "Thut", "Sme", "Set"]

e_m = ["ph", "phth", "ho", "huti", "nou", "shep", "kn",
       "de", "phr", "khm", "kh", "dj", "mh", "khk", "er", "utme",
       "hm", "khe", "nemh", "khn", "fer", "f", "pta", "djm", "fer",
       "auten", "nkhk", "utmose", "ankh", "ankha", "kham"]

e_s = ["are", "eru", "ys", "ek","esu", "et", "em", "at", "ten",
       "eb", "ut", "uti", "hotep", "ho", "tep", "are", "aten",
       "tari", "ti", "fre", "fre", "tem", "fra", "mose", "mon", "ket",
       "kh", "det", "tep", "th", "nut", "sis", "mheb", "nkh",
       "ankh", "ptah","met", "kmet", "djmet", "bek", "feru", "khamon",
       "sret", "sobek", "khufu", "psut"]

#### ROMAN ####
r_p  = ["Luc", "Val", "Mar", "Ovi", "Gai", "Hel", "Agrip",
        "C", "Tac", "Ta", "Ae", "Au", "Aure", "Cae", "Cl", "Fla",
        "Gl", "Gn", "Iu", "Ju", "Jul", "Liv", "Ma", "Marc",
        "Max", "Maxi", "Por", "Prisc", "Quint", "Sci", "Sev",
        "Silv", "Tit", "Va", "Vi", "Val", "Var"]
r_m  = ["an", "in", "ian", "ae", "an", "en", "ar", "qu", "v",
        "c", "n", "sc", "t","p", "aia", "l", "lian", "mil", "qui", "ll",
        "ss", "ssian", "ian", "aud", "au", "ci", "m", "iab", "ael",
        "up", "rc", "nl", "cell", "ae", "ver", "lp", "rqu", "quin",
        "nti", "nt", "ell", "ni"]
r_s  = ["ia","ius", "us", "ina", "ella", "a", "iana", "aeus"]

#### GREECE ####
g_p  = [ "Lys", "Kal", "Ari","Rho", "Kor", "Arte", "Ae", "Ch", "Chl",
        "Deme", "Dem", "Tel", "Tele", "Thou", "Eu", "A", "Tha", "Da",
        "Her", "Hesp", "Ny", "Ph", "Per", "Pel", "Phi", "Th", "Theo",
         "Tha", "Xa", "Xe", "Xeno", "Zo", "Ze", "Louk"]
g_m    = ["er", "in", "th", "t", "kr", "d", ", tr", "k", "ll"
          "ph", "gor", "ch", "nt", "sch", "l", "ga", "pe", "ng", "ei"
          "el", "gath", "x", "xan", "lk", "ia", "mp", "el", "rk"
          "ari", "sth", "ian", "nth", "pho", "phe" "sthe", "hem", "rg"
          "thy", "pp", "mach", "ymph", "mph", "meni", "ych", "ou"]
g_s   = ["ius", "ias", "is", "as",  "ache", "one", "is", "des", "os"
         "ios", "us", "es", "eas", "eus", "ate", "ates", "ice", "sos"
         "os", "oë", "eus", "sia", "kles", "ion", "e", "ton", "eon"
         "phon", "es", "d", "od", "iod", "ene", "ates", "kles", "che"]

#### NEAR EAST ####
ne_p = ["Ha", "Hamm", "Has", "Nab", "Nabo", "Nabu", "Esar", "E",
        "Bel", "Sar", "Sha", "Shal", "En", "Enh",]
ne_m = ["mmu", "dru", "apla", "pol", "mmur", "nn", "zz", "mil",
        "sharra", "sha", "sh", "kk", "ira", "ban", "shur", "ch",
        "dn", "adn", "lm", "man", "a", "shl", "nhe", "dd", "rh"  ]
ne_s = ["ssar", "assar", "usur", "nna", "car", "ar", "zzar"
        "don", "rri", "i", "nezzar", "ton", "gon", "mis", "amis"
        "bi", "neser", "mat", "at", "ltum", "tum", "rra", "shua"]

# create class instance for each culture
Egypt = Culture(e_p, e_m, e_s, "Egyptian")
Rome = Culture(r_p, r_m, r_s, "Roman")
Greece = Culture(g_p, g_m, g_s, "Greek")
Near_East = Culture(ne_p, ne_m, ne_s, "Near East")

# list of all cultures/classes
all_cultures = [Egypt, Rome, Greece, Near_East]


# Main
AFTER pressing "Run All", press the play button below.

In [ ]:
# <- Hover over me!! Click that button!!

for i in range(5):
  generateName(all_cultures)

Anumilaequ
  Egyptian - Roman - Roman - Roman
  Anu - mil - ae - qu

Lucup
  Roman - Roman
  Luc - up

Juler
  Roman - Egyptian
  Jul - er

Vaqui
  Roman - Roman
  Va - qui

Valer
  Roman - Egyptian
  Val - er



# hide
don't worry about this

In [ ]:
cult = cultures[3]
tester = cult.pre
arr = tester.get("vc")
all = tester.get("all")


print(cult.name)
print(all)
print(arr)
print()

result=""

for i, idx in enumerate(arr):
  result = result + ", "+ all[idx]

result = result.removeprefix(", ")
print(result)


NameError: name 'cultures' is not defined

In [ ]:
# old verison
def generateName(cultures):

  cult = random.choice(cultures)

  name = ""

  p = random.choice(cult.pre.get("all"))


  debug1 = (cult.name)
  debug2 = (p)

  ### PREFIX
  name = p;

  if isVowel(p[len(p) - 1]):
    next = 'c'
  else:
    next = 'v'

  ### MIDDLE
  m = ""
  if random.choice([True, False]):
    cult = random.choice(cultures)

    if next == 'c':
      idx = random.choice(cult.mid.get("c_any"))
    else:
      idx = random.choice(cult.mid.get("v_any"))


    m = cult.mid["all"][idx]

    if isVowel(m[len(m) - 1]):
      next = 'c'
    else:
      next = 'v'

    debug1 = (debug1+ " - " + cult.name)
    debug2 = (debug2+ " - " + m)

  ### SUFFIX
  cult = random.choice(cultures)

  if next == 'c':
    idx = random.choice(cult.suf.get("c_any"))
  else:
    idx = random.choice(cult.suf.get("v_any"))

  s = cult.suf.get("all")[idx]

  debug1 = (debug1+ " - " + cult.name)
  debug2 = (debug2+ " - " + s)

  ### CONSTRUCT
  if m == "":
      name = p + s
  else:
      name = p + m + s

  ### DEBUG
  print(name)
  print("  "+debug1)
  print("  "+debug2 + "\n")

